In [23]:
#pip install openai

In [1]:
import pandas as pd

df = pd.read_csv('pt_question_answers_updated.csv')
df

,pt_post_id,pt_post_type_id,pt_accepted_answer_id,pt_creation_date,pt_score,pt_title,pt_body,pt_tags,pt_parent_id,context,pt_answer,question
0,34750268,1,34762233.0,2016-01-12T17:36:25.473,9,Extracting the top-k value-indices from a 1-D ...,<p>Given a 1-D tensor in Torch (<code>torch.Te...,<python><lua><pytorch><torch>,NaN,<p>Just loop through the tensor and run your c...,"<p>As of pull request <a href=""https://github....",Extracting the top-k value-indices from a 1-D ...
1,38543850,1,38676842.0,2016-07-23T16:15:43.967,40,How to Display Custom Images in Tensorboard (e...,"<p>The <a href=""https://github.com/tensorflow/...",<python><tensorflow><matplotlib><pytorch><tens...,NaN,<p>It is quite easy to do if you have the imag...,<p>It is quite easy to do if you have the imag...,How to Display Custom Images in Tensorboard (e...
2,41767005,1,43824857.0,2017-01-20T15:22:08.063,11,Python wheels: cp27mu not supported,"<p>I'm trying to install pytorch (<a href=""htt...",<python><linux><unicode><pytorch>,NaN,<p>This is exactly that. \nRecompile python un...,<p>This is exactly that. \nRecompile python un...,Python wheels: cp27mu not supported <p>I'm try...
3,41861354,1,54261158.0,2017-01-25T20:45:35.297,8,Loading Torch7 trained models (.t7) in PyTorch,<p>I am using Torch7 library for implementing ...,<python><lua><pytorch><torch><pre-trained-model>,NaN,<p>The correct function is <code>load_lua</cod...,<p>As of PyTorch 1.0 <code>torch.utils.seriali...,Loading Torch7 trained models (.t7) in PyTorch...
4,41924453,1,42054194.0,2017-01-29T18:31:24.687,65,PyTorch: How to use DataLoaders for custom Dat...,<p>How to make use of the <code>torch.utils.da...,<python><torch><pytorch>,NaN,"<p>Yes, that is possible. Just create the obje...","<p>Yes, that is possible. Just create the obje...",PyTorch: How to use DataLoaders for custom Dat...
...,...,...,...,...,...,...,...,...,...,...,...,...
10758,74612146,1,NaN,2022-11-29T09:54:30.430,0,Is it possible to perform quantization on dens...,<p>I have been trying to performing quantizati...,<machine-learning><pytorch><artificial-intelli...,NaN,<p>Here's how to do this on DenseNet169 from t...,<p>Here's how to do this on DenseNet169 from t...,NaN
10759,74637151,1,NaN,2022-12-01T05:08:37.150,1,"Why when the batch size increased, the epoch t...",<p>Epoch time means the time required to train...,<deep-learning><pytorch>,NaN,"<p>As you already noticed, there are many fact...","<p>As you already noticed, there are many fact...",NaN
10760,74642594,1,NaN,2022-12-01T13:23:27.277,0,Why does StableDiffusionPipeline return black ...,"<p>I am using the <a href=""https://github.com/...",<python><pytorch><apple-m1><huggingface-transf...,NaN,<p>Apparently it is indeed an Apple Silicon (M...,<p>Apparently it is indeed an Apple Silicon (M...,NaN
10761,74671399,1,NaN,2022-12-03T22:46:46.443,1,Locating tags in a string in PHP (with respect...,<p>I want to create a function that labels the...,<php><string><pytorch><label><italics>,NaN,<p>I think I've got something. How about this:...,<p>I think I've got something. How about this:...,NaN


### get url given question id:

In [2]:
import requests
from bs4 import BeautifulSoup

def get_question_url(question_id):
    url = f"https://stackoverflow.com/questions/{question_id}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    question_link = soup.find("a", {"class": "question-hyperlink"})
    if question_link is not None:
        return question_link.get("href")
    else:
        canonical_link = soup.find("link", {"rel": "canonical"})
        if canonical_link is not None:
            return canonical_link['href']
        else:
            return None


### using url extract question and answer using openai:

In [3]:
import requests
import time
from tqdm import tqdm
import os
import openai

# Insert your openai key here
api_key = ''

def get_qa(context):
    try:
        completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo", api_key = api_key,
                messages=[
                    {"role": "user", "content": context}
                  ]
                )

        qa = completion.choices[0].message.content

    except requests.exceptions.RequestException as e:
        print(f'Request to {url} failed with error: {str(e)}.')
        print(f'Waiting for 3 minutes before trying again...')
        time.sleep(180)
    
    return qa


In [7]:
num_of_posts = 10

In [8]:
questions = []
answers = []
for question_id in tqdm(df.pt_post_id[:num_of_posts]):
    full_url = get_question_url(question_id)
    context = 'extract question and answer from following link: {}'.format(full_url)
    qa = get_qa(context)
    question, answer = qa.split('Answer:')
    question = question.split('Question:')[-1]
    question = question.strip()
    answer = answer.strip()
    questions.append(question)
    answers.append(answer)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [03:50<00:00, 23.05s/it]


In [9]:
df1 = pd.DataFrame({
    "questions": questions,
    "answers": answers
})
df1

,questions,answers
0,Extracting the top-k value indices from a 1-D ...,To extract the top-k values and indices from a...
1,"How to display custom images in Tensorboard, e...",One way to display custom images in Tensorboar...
2,Python Wheels cp27mu not supported,"The ""cp27mu"" suffix stands for a specific vers..."
3,Loading Torch7 trained models (.t7) in PyTorch,PyTorch can load pre-trained Lua Torch model u...
4,PyTorch - How to use dataloaders for custom da...,To use PyTorch's dataloader with custom datase...
5,What does view() do in PyTorch?,"In PyTorch, view() is a method used to reshape..."
6,How do I print the model summary in PyTorch?,You can use the summary function provided by t...
7,How do I save a trained model in PyTorch?,"In PyTorch, a trained model can be saved by us..."
8,What is L1 and L2 regularization in PyTorch?,L1 and L2 regularization are techniques to pre...
9,How can I install torchtext?,"To install torchtext, you can use pip as follo..."


In [11]:
df1.to_csv('qa.csv')